In [97]:
%matplotlib inline
%matplotlib nbagg
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import prettytensor as pt

In [98]:
rows, cols = 70, 100
img_shape = (rows, cols)
img_flat_size = rows * cols
img_classes = 2
img_channels = 3

phase = pt.Phase.train

In [99]:
resized_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/havss-tf/ISIC-2017/resized/"

In [100]:
isic_pickle_file = resized_folder_path + "ISIC.pickle"
with open(isic_pickle_file, "rb") as f:
    save = pickle.load(f)
    train_dataset =  save["train_dataset"]
    train_labels =  save["train_labels"]
#     validation_dataset =  save["validation_dataset"]
#     validation_labels =  save["validation_labels"]
    test_dataset =  save["test_dataset"]
    test_labels =  save["test_labels"]
    del save
    print("Training Set {0} {1}".format(train_dataset.shape, train_labels.shape))
#     print("Validation Set {0} {1}".format(validation_dataset.shape, validation_labels.shape))    
    print("Test Set {0} {1}".format(test_dataset.shape, test_labels.shape))
print test_labels[0:10, :]
dataset_info = np.sum(test_labels[:, :], axis = 0)
print("Test Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(dataset_info[0], dataset_info[1]))
# dataset_info = np.sum(validation_labels[:, :], axis = 0)
# print("Validation Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(dataset_info[0], dataset_info[1]))
dataset_info = np.sum(train_labels[:, :], axis = 0)
print("Train Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(dataset_info[0], dataset_info[1]))
dataset_info = []

Training Set (37904, 70, 100, 3) (37904, 2)
Test Set (6152, 70, 100, 3) (6152, 2)
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]]
Test Dataset Info : [1, 0] = 2786.0, [0, 1] = 3366.0
Train Dataset Info : [1, 0] = 20612.0, [0, 1] = 17292.0


In [101]:
tf.reset_default_graph() 

In [102]:
# input_x = tf.placeholder(tf.float32, [None, img_flat_size], name = "input_x") 
input_layer = tf.placeholder(tf.float32, [None, rows, cols, img_channels], name = "input_layer")
y_true = tf.placeholder(tf.float32, [None, img_classes], name = "y_true")
y_true_cls = tf.argmax(y_true, dimension = 1)

In [103]:
x_pretty = pt.wrap(input_layer)
with pt.defaults_scope(activation_fn = tf.nn.relu):
    y_pred, loss = x_pretty.\
        conv2d(kernel = 5, depth = 16, name = "conv_layer_1").\
        max_pool(kernel = 2, stride = 2).\
        dropout(keep_prob=0.8, phase=phase).\
        conv2d(kernel = 5, depth = 32, name = "conv_layer_2").\
        max_pool(kernel = 2, stride = 2).\
        dropout(keep_prob=0.8, phase=phase).\
        conv2d(kernel = 5, depth = 64, name = "conv_layer_3").\
        max_pool(kernel = 2, stride = 2).\
        flatten().\
        fully_connected(size = 256, name = "fc_layer_2", activation_fn = tf.nn.relu).\
        softmax_classifier(num_classes = img_classes, labels = y_true)

In [104]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

In [105]:
y_pred_cls = tf.argmax(y_pred, dimension = 1)

In [106]:
correct_pred = tf.equal(y_true_cls, y_pred_cls)

In [107]:
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [108]:
train_batch_size = 32
prev_train_acc = 0
prev_train_acc_step = 0
min_train_acc_step = 1000
offset = 0

In [109]:
# num_steps = 12000
# with tf.Session() as sess:
#     tf.global_variables_initializer().run()
#     print("Variable Initialized")
#     for step in range(num_steps):
#         offset = (step * train_batch_size) % (train_labels.shape[0] - train_batch_size)
# #         print offset
#         batch_train = train_dataset[offset:(offset + train_batch_size), :, : ,:]
#         batch_labels = train_labels[offset:(offset + train_batch_size), :]
#         feed_dict_train = {
#             input_layer: batch_train,
#             y_true: batch_labels
#         }
        
#         _, l = sess.run([optimizer, loss], feed_dict = feed_dict_train)
#         print("Step : {0} - Loss : {1}".format(step, l))
        

#         if step % 200 :
#             feed_dict_valid = {
#                 input_layer: validation_dataset,
#                 y_true: validation_labels
#             }

#             valid_acc = sess.run([accuracy], feed_dict = feed_dict_valid)
#             print("Step : {0} - Validation Accuracy : {1} ".format(step, train_loss))
            
#     phase = pt.Phase.test    
#     feed_dict_test = {
#         input_layer: test_dataset,
#         y_true: test_labels
#     }
#     true_value_class, acc, pred_value, pred_value_class = sess.run([y_true_cls, accuracy, y_pred, y_pred_cls], feed_dict = feed_dict_test)
#     print("Accuracy {:.2%}".format(acc))
#     print("True Value Class")
#     print true_value_class[0:10]
#     print("Pred Value")
#     print pred_value[0:20]
#     print("Pred Value Class")
#     print pred_value_class

In [110]:
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

In [135]:
def runModel(epoch):
    with tf.Session() as sess:
        sess.run(init_op)
        print("Variables Initialized")
        total_data_len = train_labels.shape[0]
        print total_data_len
        num_step = int(total_data_len / train_batch_size)
        print("Batch Size : {0}".format(train_batch_size))
        print("Starting Training")
        for i in range(epoch):
            print("Epoch - {0} ".format(i))
            for step in range(num_step):
                offset = (step * train_batch_size) % (total_data_len - train_batch_size)
                batch_train = train_dataset[offset:(offset + train_batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + train_batch_size), :]
                feed_dict_train = {
                    input_layer: batch_train,
                    y_true: batch_labels
                }

                _, train_loss = sess.run([optimizer, loss], feed_dict = feed_dict_train)
                print("Step : {0} - Training Loss : {1}".format(step, train_loss))
                
#                 if step % 200 :
#                     feed_dict_valid = {
#                         input_layer: validation_dataset[:2000,:,:,:],
#                         y_true: validation_labels[:2000,:]
#                     }
                    
#                     valid_acc = sess.run([accuracy], feed_dict = feed_dict_valid)
#                     print("Step : {0} - Validation Accuracy : {1} ".format(step, train_loss))
                    
#                     if valid_acc > prev_train_acc:
#                         print("Imporvement Found : {0}, Step Difference : {1}".format(valid_acc - prev_train_acc, step - prev_train_acc_step))
#                         prev_train_acc_step = step
#                         prev_train_acc = valid_acc
#                     else:
#                         if prev_train_acc_step - step > min_train_acc_step:
#                             print("No Improvement Found : Step Difference : {0}".format(step - prev_train_acc_step))
#                             print("Therefore Ending Training")
#                             break

        save_path = saver.save(sess, "/home/openroot/tensorboard/isic/model.ckpt")
        print("Model saved in file: %s" % save_path)
        train_dataset = []
        train_labels = []
        
        print("Training Completed!!!")
        print("Staring Testing Phase")
        phase = pt.Phase.test    
        
        total_test_data_len = test_labels.shape[0]
        final_test_pred = np.array([])
        print total_test_data_len
        for i in range(total_test_data_len):        
            feed_dict_test = {
                input_layer: test_dataset[i,:,:,:],
                y_true: test_labels[i,:]
            }      
            test_batch_pred = sess.run(y_pred, feed_dict = feed_dict_test)
            if i % 200:
                print("Training Step : {0}, Pred : {}, Shape : {}". format(i, test_batch_pred, test_batch_pred.shape))
#                 print test_batch_pred
#                 print test_batch_pred.shape
            final_test_pred = np.append(final_test_pred, test_batch_pred)

        print "------------ Shape"
    
        final_test_pred = np.reshape(final_test_pred, (-1, 2))
        print final_test_pred.shape
        print final_test_pred[0:10,:]
        
        print "------------ Pred Test"
        
        final_test_pred_cls = np.argmax(final_test_pred, axis = 1)
        print final_test_pred_cls.shape
        print final_test_pred_cls[0:10]
        
        print "------------ True Test"
        
        final_test_true_cls = np.argmax(test_labels, axis = 1)
        print final_test_true_cls.shape
        print final_test_true_cls[0:10]   
        
        print "------------ Acc Test"
        final_correct_test = np.equal(final_test_pred_cls[0:10], final_test_true_cls[0:10]).astype(np.float32)
        print final_correct_test
        test_acc = np.mean(final_correct_test, dtype=np.float64)
        print "Test Accuracy : {0}".format(test_acc)
        

In [136]:
runModel(10)

Variables Initialized
37904
Batch Size : 32
Starting Training
Training Completed!!!
Staring Testing Phase
6152
(32, 2)
------------
(32, 2)
[[ 0.47484732  0.52515268]
 [ 0.00851193  0.9914881 ]
 [ 0.02398085  0.97601908]
 [ 0.55799192  0.44200808]
 [ 0.01327656  0.98672348]
 [ 0.52397537  0.47602466]
 [ 0.26622349  0.73377651]
 [ 0.64744401  0.35255608]
 [ 0.5         0.5       ]
 [ 0.45266917  0.54733086]]
------------ Pred
(32,)
[1 1 1 0 1 0 1 0 0 1]
------------ True
(6152,)
[1 1 0 0 0 0 1 0 0 1]
------------ Acc
[ 1.  1.  0.  1.  0.  1.  1.  1.  1.  1.]
Acc : 0.8
